In [176]:
import pandas as pd
import requests
import yfinance as yf
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [177]:
class Accion:
    def __init__(self, arrayaccion) -> None:
        pass

class Origen:
    def __init__(self):
       self.__abrirColeccion()

    def __abrirColeccion(self):
        client = MongoClient('localhost')
        bd = client['cotizaciones']
        self.coleccion = bd[self.__class__.__name__]
        
    def _guardarEnBD(self, dataFrame):
        self.coleccion.drop()
        self.coleccion.insert_many(dataFrame.to_dict('records'))
        
    def actualizarBD(self):
        pass

    def visualizarBD(self):
        for documento in self.coleccion.find({}):
            print(documento)
            
    def visualizarDF(self):
        df = self.__documentADF(self.coleccion.find({}))
        print (df)

    def __documentADF(self, listColeccion):
        df = pd.DataFrame(list(listColeccion))
        return df.drop(['_id'], axis=1)
        
    def __cotizacionMinima(self):
        pass
    def __cotizacionMaxima(self):
        pass

    def filtrarAcciones(self, nombreAccion):
        accion = self.coleccion.find({self.campo: nombreAccion})
        return print(accion.next())
        #return print(self.__documentADF(accion))

    def accion(self, symbol):
        return Accion(self.filtrarAcciones(symbol))

origen = Origen()
origen.actualizarBD()


In [178]:
       
class bolsaMadrid(Origen):
    campo = 'Nombre'
    
    def actualizarBD(self):
        #Hacemos la consulta a la pagina y guardamos el contenido en una variable soup
        url_page = 'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'
        page = requests.get(url_page).text 
        soup = BeautifulSoup(page, "lxml")

        #Buscamos la tabla de donde obtener los datos
        tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})

        #creamos una lista vacia que llenaremos con los datos de las tablas
        columnas = []

        #buscamos los encabezados de la tabla y los agregamos a la lista columnos (deberia ubicarse en la posicion 0)
        headers = []
        for th in tabla.find_all('th'):
            headers.append(th.text)
        columnas.append(headers)

        for tr in tabla.find_all('tr'):
            fila = []
            for td in tr.find_all('td'):
                fila.append(td.text)
            if len(fila) > 0:
                columnas.append(fila)
                
        #creamos el dataframe usando la lista columnas,pasamos el contenido (de la posicion 1 en adelante), y lo encabezados en (posicion 0)
        df = pd.DataFrame(columnas[1:], columns=columnas[0])

        #Convertirmos a float
        df = self.__convertirAFloat(df)

        #Guardamos los datos en la base de datos
        self._guardarEnBD(df)
        
    def __convertirAFloat(self, df):
        df["% Dif."] = df["% Dif."].str.replace(",", ".").astype(float)
        df["Máx."] = df["Máx."].str.replace(",", ".").astype(float)
        df["Mín."] = df["Mín."].str.replace(",", ".").astype(float)
        df["Últ."] = df["Últ."].str.replace(",", ".").astype(float)
        return df

bolsamadrid = bolsaMadrid()
bolsamadrid.actualizarBD()
bolsamadrid.visualizarDF()
bolsamadrid.filtrarAcciones(['ACCIONA', 'ACS'])

          Nombre      Últ.  % Dif.      Máx.      Mín.     Volumen  \
0        ACCIONA  166.0000    0.12  168.9000  165.9000     106.842   
1       ACERINOX   12.2450    1.74   12.3450   12.1500     883.382   
2            ACS   23.0200    1.72   23.2000   22.8000     561.232   
3           AENA  145.6500    2.82  146.3500  141.1000     108.932   
4       ALMIRALL   13.2100    2.80   13.2500   12.9500     302.296   
5        AMADEUS   58.0000    0.28   58.6000   57.4400     998.615   
6    ARCELORMIT.   29.9000    1.91   30.1450   29.6200     592.253   
7    B.SANTANDER    3.3435    2.04    3.3535    3.2755  77.143.204   
8    BA.SABADELL    0.6990    0.37    0.7184    0.6970  27.723.034   
9      BANKINTER    4.9090    3.09    4.9570    4.7850   3.618.007   
10          BBVA    6.2790    3.58    6.2920    6.0300  19.040.383   
11     CAIXABANK    2.4870    0.08    2.5100    2.4700  19.265.473   
12       CELLNEX   53.1400   -0.08   53.4800   52.5600     798.991   
13  CIE AUTOMOT.   2

StopIteration: 

In [ ]:
class yfinance():

    def __init__(self):
       pass
    def obtenerDatos(acciones):
        df_list = list()
        for ticker in acciones:
            data = yf.download(ticker, group_by='ticker', period='1d')
            data['ticker'] = ticker  
            df_list.append(data)

        df = pd.concat(df_list)

        # save to csv
        df.to_csv('yahoo_finance.csv')

class investpy():

   def __init__(self):
      pass

   def extraerDatos():
      pass


class Cotizaciones(bolsaMadrid, yfinance, investpy):

    def __init__(self, acciones):
        self.acciones = acciones
        self.actualizarBD()
       

    def actualizarBD(self):
        bolsaMadrid.obtenerDatos(self)
        yfinance.obtenerDatos(self.acciones)
        ##investpy.obtenerDatos(self)

    def leerDatos():
        pass

In [ ]:
acciones = ['ACCIONA', 'BBVA', 'GRIFOLS CL.A', 'PEPITO', 'REPSOL', 'ENDESA']
cotizaciones = Cotizaciones(acciones)
cotizaciones.mostrarDatosEnPantalla()


AttributeError: type object 'bolsaMadrid' has no attribute 'obtenerDatos'